In [1]:
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
import plotly.graph_objects as go
import os
import numpy as np
from tqdm import tqdm  
from keras.models import Model
from keras.layers import Input, Dense, Reshape, Flatten, Activation
from keras.layers import Embedding, Lambda, Concatenate, Add
from keras.layers import Conv3DTranspose, Conv3D
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
import keras.backend as K
from keras.layers import Conv2D, Conv2DTranspose, Reshape
from keras.models import Model
from keras.layers import Input, Dense, Reshape, Flatten, Activation
from keras.layers import Embedding, Lambda, Concatenate, Add
from keras.layers import GlobalAvgPool3D, Multiply
from keras.layers import Conv3DTranspose, Conv3D
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
import keras.backend as K

dataset_path = "ModelNet40"
images_path = "images"

In [2]:
def dense_layer(inp, f, act='relu', bn=True):
    initializer = act if act is not None else ''
    initializer = 'he_uniform' if initializer.find('relu') != -1 else 'glorot_uniform'
    out = Dense(f, use_bias=False, kernel_initializer=initializer)(inp)
    if bn: out = BatchNormalization()(out)
    
    if act == 'lrelu':
        out = LeakyReLU(alpha=0.2)(out)
    elif act is not None:
        out = Activation(act)(out)
    
    return out

def conv_layer(inp, f, k=4, s=2, p='same', act='relu', bn=True, transpose=False,
               se=False, se_ratio=16):
    initializer = act if act is not None else ''
    initializer = 'he_uniform' if initializer.find('relu') != -1 else 'glorot_uniform'
    fun = Conv3DTranspose if transpose else Conv3D
    out = fun(f, k, strides=s, padding=p, use_bias=False, kernel_initializer=initializer)(inp)
    if bn: out = BatchNormalization()(out)
    
    if act == 'lrelu':
        out = LeakyReLU(alpha=0.2)(out)
    elif act is not None:
        out = Activation(act)(out)

    # squeeze and excite
    if se:
        out_se = GlobalAvgPool3D()(out)
        r = f // se_ratio if (f // se_ratio) > 0 else 1
        out_se = Reshape((1, 1, f))(out_se)
        out_se = Dense(r, use_bias=False, kernel_initializer='he_uniform',
                       activation='relu')(out_se)
        out_se = Dense(f, use_bias=False, activation='sigmoid')(out_se)
        out = Multiply()([out, out_se])
    
    return out

In [3]:
def _generator_v_3d(dict_size):
    # inputs
    labels = Input(shape=(1,))
    image_inp = Input(shape=(128, 128, 1))
    image = Lambda(lambda x: K.expand_dims(x))(image_inp)
    
    # label embedding
    embs = Embedding(dict_size, 64, input_length=1)(labels)
    embs = Flatten()(embs)
    embs = dense_layer(embs, 1024)
    
    # conv layers for image processing
    image_conv1 = Conv2D(32, kernel_size=5, strides=1, padding="same", activation="relu")(image)
    image_conv2 = Conv2D(32, kernel_size=3, strides=1, padding="same", activation="relu")(image_conv1)
    image_conv3 = Conv2D(64, kernel_size=3, strides=1, padding="same", activation="relu")(image_conv2)
    image_conv4 = Conv2D(128, kernel_size=3, strides=1, padding="same", activation="relu")(image_conv3)
    image_conv5 = Conv2D(256, kernel_size=3, strides=1, padding="same", activation="relu")(image_conv4)
    
    image_flatten = Flatten()(image_conv5)
    
    # Merge label embedding and processed image
    mix = Concatenate()([image_flatten, embs])
    mix = dense_layer(mix, 1024)
    mix = dense_layer(mix, 2*2*2*256)
    mix = Lambda(lambda x: K.reshape(x, (-1, 2, 2, 2, 256)))(mix)

    # Transpose convolution layers
    out = Conv3DTranspose(256, kernel_size=(3, 3, 3), strides=(2, 2, 2), padding="same", activation="relu")(mix)
    out = Conv3DTranspose(128, kernel_size=(3, 3, 3), strides=(2, 2, 2), padding="same", activation="relu")(out)
    out = Conv3DTranspose(64, kernel_size=(3, 3, 3), strides=(2, 2, 2), padding="same", activation="relu")(out)
    out = Conv3DTranspose(32, kernel_size=(3, 3, 3), strides=(2, 2, 2), padding="same", activation="relu")(out)
    
    # Output layer
    out = Conv3D(1, kernel_size=(3, 3, 3), padding="same", activation="tanh")(out)
    
    return Model((image_inp, labels), out)
# Discriminator models:

def _discriminator(dict_size, se=False):
    # inputs
    labels = Input(shape=(1,))
    voxels_inp = Input(shape=(128,128,1))
    voxels = Lambda(lambda x: K.expand_dims(x))(voxels_inp)
    
    # label embedding
    embs = Embedding(dict_size, 64, input_length=1)(labels)
    embs = Flatten()(embs)
    embs = dense_layer(embs, 1024, act='lrelu', bn=False)
    
    # conv layers
    out = conv_layer(voxels, 32, 5, 1, act='lrelu', bn=False, se=se)
    out = conv_layer(out, 32, act='lrelu', bn=False, se=se)
    out = conv_layer(out, 64, act='lrelu', bn=False, se=se)
    out = conv_layer(out, 128, act='lrelu', bn=False, se=se)
    out = conv_layer(out, 256, act='lrelu', bn=False, se=se)
    out = Flatten()(out)
    out = dense_layer(out, 1024, act='lrelu', bn=False)
    out = Concatenate()([out, embs])
    out = dense_layer(out, 1024, act='lrelu', bn=False)
    out = dense_layer(out, 512, act='lrelu', bn=False)
    out = dense_layer(out, 1, act=None, bn=False)
    
    return Model((voxels_inp, labels), out)

def make_discriminator(dict_size, model_type):
    model = {
        'voxels-v': _discriminator(dict_size),
        'voxels-u': _discriminator(dict_size),
        'voxels-use': _discriminator(dict_size, se=True)
    }
    
    return model[model_type]

def make_generator(dict_size, model_type):
    model = {
        'voxels-v': _generator_v(dict_size),
        'voxels-u': _generator_u(dict_size),
        'voxels-use': _generator_u(dict_size, se=True)
    }
    
    return model[model_type]

In [4]:
def load_model_files(folder_path):
    model_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".off"):  # Adjust the file extension as needed
                model_files.append(os.path.join(root, file))
    return model_files

def load_model_data(file_path):
    # Implement your code to load 3D model data from the file
    # You might use the readOff function or any other method
    # Return the loaded model data
    pass

def generate_dataset(data_folder):
    X_train = []
    X_test = []

    # Iterate over the object class folders
    for class_folder in tqdm(os.listdir(data_folder)):
        if class_folder.lower() == 'bathtub':  # Skip the "bathtub" folder
            continue

        class_path = os.path.join(data_folder, class_folder)

        # Skip non-directory items
        if not os.path.isdir(class_path):
            continue

        # Load model files for the current class
        model_files = load_model_files(class_path)

        # Split the model files into train and test sets
        split_index = int(len(model_files) * 0.8)  # Adjust the split ratio as needed
        train_files = model_files[:split_index]
        test_files = model_files[split_index:]

        # Load train set
        for file in train_files:
            model_data = load_model_data(file)
            X_train.append(model_data)

        # Load test set
        for file in test_files:
            model_data = load_model_data(file)
            X_test.append(model_data)

    # Convert lists to numpy arrays
    X_train = np.array(X_train)
    X_test = np.array(X_test)

    return X_train, X_test

In [5]:
def readOff(filename):
    f = open(filename)
    f.readline()
    nvertices, nfaces, nedges = map(int, f.readline().split())
    vertices = []
    for _ in range(nvertices):
        vertices.append(list(map(float, f.readline().strip().split())))
    vertices = np.array(vertices)

    triangles = []
    for _ in range(nfaces):
        face = list(map(int, f.readline().strip().split()))
        ntriangles, verts = face[0] - 3 + 1, face[1:]
        for n in range(ntriangles):
            triangles.append([verts[0], verts[1 + n], verts[2 + n]])
    triangles = np.array(triangles)

    return vertices, triangles

def save_png(off_filename, png_filename):
    vertices, faces = readOff(off_filename)

    x, y, z = vertices.T
    I, J, K = faces.T

    mesh = go.Mesh3d(
        x=-x,
        y=y,
        z=z,
        i=I,
        j=J,
        k=K,
        name='',
        showscale=False,
        color='brown'
    )

    fig = go.Figure(data=mesh)
    fig.update_layout(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False)
        )
    )
    fig.write_image(png_filename)
    # print(f"Image saved as {png_filename}")

In [6]:
def generate_and_save_data(data_folder, output_folder):
    for class_folder in os.listdir(data_folder):
        if class_folder.lower() == 'bathtub':  # Skip the "bathtub" folder
            continue

        class_path = os.path.join(data_folder, class_folder)

        # Skip non-directory items
        if not os.path.isdir(class_path):
            continue

        output_class_folder = os.path.join(output_folder, class_folder)
        os.makedirs(output_class_folder, exist_ok=True)

        for split_folder in ['train', 'test']:
            split_path = os.path.join(class_path, split_folder)

            for model_file in os.listdir(split_path):
                if model_file.endswith(".off"):  # Adjust the file extension as needed
                    off_file_path = os.path.join(split_path, model_file)
                    png_file_name = f"{model_file[:-4]}_{split_folder}.png"
                    png_file_path = os.path.join(output_class_folder, png_file_name)

                    save_png(off_file_path, png_file_path)

In [7]:
X_train, X_test = generate_dataset(dataset_path)
#generate_and_save_data(dataset_path, images_path)

100%|██████████| 40/40 [00:00<00:00, 597.03it/s]


In [8]:
dict_size = 40-1  

discriminator = make_discriminator(dict_size, 'voxels-use')
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5), metrics=['accuracy'])

discriminator.trainable = False

generator = _generator_v_3d(dict_size)
gan_input = [Input(shape=(128, 128, 1)), Input(shape=(1,))]
generated_voxel = generator(gan_input)
validity = discriminator([generated_voxel, gan_input[1]])
gan = Model(gan_input, validity)
gan.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))

NameError: name 'GlobalAvgPool3D' is not defined

In [ ]:
import numpy as np

epochs = 10000  # Adjust as needed
batch_size = 64  # Adjust as needed

for epoch in range(epochs):
    # ---------------------
    #  Train Discriminator
    # ---------------------

    # Select a random batch of images
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    real_images = X_train[idx]

    # Generate a batch of new images
    labels = np.random.randint(0, dict_size, batch_size).reshape((-1, 1))
    generated_images = generator.predict([real_images, labels])

    # Train the discriminator
    d_loss_real = discriminator.train_on_batch([real_images, labels], np.ones((batch_size, 1)))
    d_loss_fake = discriminator.train_on_batch([generated_images, labels], np.zeros((batch_size, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # ---------------------
    #  Train Generator
    # ---------------------

    # Generate a batch of new images
    labels = np.random.randint(0, dict_size, batch_size).reshape((-1, 1))
    valid_y = np.array([1] * batch_size).reshape((-1, 1))

    # Train the generator
    g_loss = gan.train_on_batch([real_images, labels], valid_y)

    # Print progress
    if epoch % 100 == 0:
        print(f"{epoch} [D loss: {d_loss[0]} | D accuracy: {100 * d_loss[1]}] [G loss: {g_loss}]")

In [ ]:
test_labels = np.random.randint(0, dict_size, batch_size).reshape((-1, 1))
generated_images = generator.predict([X_test, test_labels])

generator.save('generator_model.h5')
discriminator.save('discriminator_model.h5')